In [ ]:
import pandas as pd

df=pd.read_csv("census_2011.csv")
a=df.isnull().sum()

# Task 1: Rename the Column names
change_column_name={
  'State name': 'State/UT',
  'District name': 'District',
  'Male_Literate': 'Literate_Male',
  'Female_Literate': 'Literate_Female',
  'Rural_Households': 'Households_Rural',
  'Urban_ Households': 'Households_Urban',
  'Age_Group_0_29': 'Young_and_Adult',
  'Age_Group_30_49': 'Middle_Aged',
  'Age_Group_50': 'Senior_Citizen',
  'Age not stated': 'Age_Not_Stated'
}
df.rename(columns=change_column_name,inplace=True)

# Task 2: Rename State/UT Names
def Capitalization(df):
    for i in range(len(df)):
        x = df.loc[i,'State/UT'].lower().split()
        for j in range(len(x)):
            if x[j] != 'and':
                x[j] = x[j].capitalize()
        df.loc[i,'State/UT'] = " ".join(x)

Capitalization(df)
print(df)

# Task 3: New State/UT formation
with open('Telangana.txt', 'r') as telangana:
    telangana_districts = telangana.read()
telangana_districts = telangana_districts.split("\n")

# setting the values as Telangana if the districts are in the telangana_districts list
def telangana(df):
    for i in range(len(df)):
        if df.loc[i,'District'] in telangana_districts:
            df.loc[i,'State/UT'] = "Telangana"

telangana(df)

# setting the values as Ladakh if the districts are in the ladakh list
def ladakh(df):
    ladakh = ['Leh', 'Kargil']
    for district in ladakh:
        for i in range(len(df)):
            if district in df.loc[i,'District']:
                df.loc[i,'State/UT'] = "Ladakh"

ladakh(df)

# Task 4: Find and process Missing Data

# Finding and storing the Percentage of missing data before cleaning for each column in a separate dictionary.
def missingDataPercent(df) :
    missing_data = {}
    columns = df.columns
    for i in columns:
        count = 0
        count += len(list(filter(lambda x : x == True, list(df[i].isnull()))))
        missing_data[i] = round((count * 100)/len(df), 2)

    return missing_data

missing_data_before_cleaning = missingDataPercent(df)

# Function to return if the cell contains null value or not 
def null_values_filter(df,i,column):
    return list(df.loc[[i],column].isnull())[0]

# Filling up the Population column
def fill_population(df):
    for i in range(len(df)):
        if null_values_filter(df,i,'Population') == True:
            if null_values_filter(df,i,'Male') == null_values_filter(df,i,'Female') == False:
                df.loc[i,"Population"] = int(df.loc[i,"Male"] + df.loc[i, "Female"])
            elif null_values_filter(df,i,'Young_and_Adult') == null_values_filter(df,i,'Middle_Aged') == null_values_filter(df,i,"Senior_Citizen") == null_values_filter(df,i,'Age_Not_Stated') == False:
                df.loc[i,'Population'] = int(df.loc[i,"Young_and_Adult"] + df.loc[i,"Middle_Aged"] + df.loc[i, "Senior_Citizen"] + df.loc[i,"Age_Not_Stated"])
            elif null_values_filter(df,i,'Hindus') == null_values_filter(df,i,'Muslims') == null_values_filter(df,i,'Christians') == null_values_filter(df,i,'Sikhs') == null_values_filter(df,i,'Buddhists') == null_values_filter(df,i,'Jains') == null_values_filter(df,i,'Others_Religions') == null_values_filter(df,i,'Religion_Not_Stated') == False:
                df.loc[i,'Population'] = int(df.loc[i,'Hindus'] + df.loc[i,'Muslims'] + df.loc[i,'Christians'] + df.loc[i,'Sikhs'] + df.loc[i,'Buddhists'] + df.loc[i,'Jains'] + df.loc[i,'Others_Religions'] + df.loc[i,'Religion_Not_Stated'])
            elif null_values_filter(df,i,'Workers') == null_values_filter(df,i,'Non_Workers') == False:
                df.loc[i,"Population"] = int(df.loc[i,'Workers'] + df.loc[i,'Non_Workers'])

fill_population(df)

# # Calculate a percentage of the initial sum of each column with missing values 
missing_initial = (df.isnull().mean() * 100).round(4)

# Filling up the male population
def fill_population_male(df) :
    for index, row in df.iterrows():
        if pd.isnull(row['Male']):
            if pd.notnull(row['Population']) and pd.notnull(row['Female']):
                df.loc[index, 'Male'] = row['Population'] - row['Female']
            elif pd.notnull(row['Population']):
                df.loc[index, 'Male'] = row['Population']

    return df

fill_population_male(df)

# Filling up the female population
def fill_population_female(df) :
    for index, row in df.iterrows():
        if pd.isnull(row['Female']):
            if pd.notnull(row['Population']) and pd.notnull(row['Male']):
                df.loc[index, 'Female'] = row['Population'] - row['Male']
            elif pd.notnull(row['Population']):
                df.loc[index, 'Female'] = row['Population']

fill_population_female(df)

# Filling up the Literate population
def fill_literate(df):  
    
  for index, row in df.iterrows():
        if pd.isnull(row['Literate']):
            if pd.notnull(row['Literate_Male']) and pd.notnull(row['Literate_Female']):
                df.loc[index, 'Literate'] = row['Literate_Male'] + row['Literate_Female']
            elif pd.notnull(row['Literate_Male']):
                df.loc[index, 'Literate'] = row['Literate_Male']
            elif pd.notnull(row['Literate_Female']):
                df.loc[index, 'Literate'] = row['Literate_Female']

  return df

fill_literate(df)

# Filling up the Literate Male population
def fill_literate_male(df):
   
    for index, row in df.iterrows():
        if pd.isnull(row['Literate_Male']):
            if pd.notnull(row['Literate']) and pd.notnull(row['Literate_Female']):
                df.loc[index, 'Literate_Male'] = row['Literate'] - row['Literate_Female']
            elif pd.notnull(row['Literate']):
                df.loc[index, 'Literate_Male'] = row['Literate']

    return df
   

fill_literate_male(df)

# Filling up the Literate Female population
def fill_literate_female(df):
   
      for index, row in df.iterrows():
        if pd.isnull(row['Literate_Female']):
            if pd.notnull(row['Literate']) and pd.notnull(row['Literate_Male']):
                df.loc[index, 'Literate_Female'] = row['Literate'] - row['Literate_Male']
            elif pd.notnull(row['Literate']):
                df.loc[index, 'Literate_Female'] = row['Literate']

      return df

fill_literate_female(df)

# Filling up the Households_rural column
def fill_households_rural(df):
     for index, row in df.iterrows():

        # Check if Households_Rural is null
        if pd.isnull(row['Households_Rural']):
            households = row['Households']
            urban_households = row['Urban_Households']
            
            # If both Households and Urban_Households are present
            if pd.notnull(households) and pd.notnull(urban_households):
                households_rural = households + urban_households
                # Ensure no negative rural households
                if households_rural >= 0:
                    df.at[index, 'Households_Rural'] = households_rural
                else:
                    df.at[index, 'Households_Rural'] = 0  # Set to 0 if calculation is negative

            # If only Households is present
            elif pd.notnull(households):
                # Assume all households are rural if Urban_Households is missing or negative
                if pd.isnull(urban_households) or urban_households < 0:
                    df.at[index, 'Households_Rural'] = households

            # If only Urban_Households is present and Households is not
            elif pd.notnull(urban_households):
                # Set Households_Rural to 0 when there's no total households
                df.at[index, 'Households_Rural'] = 0

     return df


df=fill_households_rural(df)


# Filling up the Households_Urban column
def fill_households_urban(df):
       
   for index, row in df.iterrows():
        if pd.isnull(row['Urban_Households']):
            if not (pd.isnull(row['Households']) or pd.isnull(row['Households_Rural'])):
                urban_households = row['Households'] - row['Households_Rural']
                if urban_households >= 0:
                    df.loc[index, 'Urban_Households'] = urban_households
            elif pd.isnull(row['Households']):
                # If 'Households' is missing, assume 'Urban_Households' is zero
                df.loc[index, 'Urban_Households'] = 0
            elif pd.isnull(row['Households_Rural']):
                # If 'Households_Rural' is missing, assume 'Urban_Households' is equal to 'Households'
                df.loc[index, 'Urban_Households'] = row['Households']

   return df
df=fill_households_urban(df)

# Filling up the Households column
def fill_households(df):
    
     for index, row in df.iterrows():
        if pd.isnull(row['Households']):
            if pd.notnull(row['Urban_Households']) and pd.notnull(row['Households_Rural']):
                households = row['Urban_Households'] + row['Households_Rural']
                if households >= 0:
                    df.loc[index, 'Households'] = households
            elif pd.notnull(row['Urban_Households']):
                # If 'Households_Rural' is missing, assume all households are urban
                df.loc[index, 'Households'] = row['Urban_Households']
     return df


df=fill_households(df)

# Calculate a percentage of the final sum of each column after filling missing values
final_sum = (df.isnull().mean() * 100).round(4)

# Create a new DataFrame to display the results
new_df=pd.DataFrame({
'Before_filling%': missing_initial,
'After_filling%': final_sum
})
a=df.isnull().sum()
print(new_df)

# Calculate a percentage of the final sum of each column after filling missing values
final_sum = (df.isnull().mean() * 100).round(4)

missing_data_after_cleaning = missingDataPercent(df)

# Creating a CSV file to store the missing data details to visualize it in the streamlit along with other queries
missing_data = {
    'Columns': df.columns, 
    'Missing_data_before_cleaning (%)' : missing_data_before_cleaning.values(), 
    'Missing_data_after_cleaning (%)' : missing_data_after_cleaning.values()
    }
missing_data_df = pd.DataFrame(missing_data)
missing_data_df.to_csv('missing_data.csv', header=True)

df.to_csv("cleandata.csv",index=False)

     District code                     State/UT                   District  \
0                1            Jammu and Kashmir                    Kupwara   
1                2            Jammu and Kashmir                     Badgam   
2                3            Jammu and Kashmir                Leh(Ladakh)   
3                4            Jammu and Kashmir                     Kargil   
4                5            Jammu and Kashmir                      Punch   
..             ...                          ...                        ...   
635            636                  Pondicherry                       Mahe   
636            637                  Pondicherry                   Karaikal   
637            638  Andaman and Nicobar Islands                   Nicobars   
638            639  Andaman and Nicobar Islands  North  AND Middle Andaman   
639            640  Andaman and Nicobar Islands              South Andaman   

     Population      Male    Female  Literate  Literate_Male  L